# Tutorial for FastHare module
By Thang N. Dinh (tndinh@vcu.edu)

The module ipmlement the FastHare algorithm in the paper
`FastHare: Fast Hamiltonian Reduction for Large-scale Quantum Annealing`, IEEE Conf. on Quantum Computer and Enginnering, 2022[PDF](https://arxiv.org/abs/2205.05004)

In [2]:
# Install required libraries
%pip install fasthare FastHareComposite 
# Install neal library for simulated annealing
%pip install dwave-neal


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 42.9 MB/s eta 0:00:00


In [3]:
import random
import neal
import dimod
import networkx as nx
from FastHareComposite import FastHareComposite

In [ ]:
def generate_random_ising(n, d, emin, emax):
  """Generate random Ising Hamiltonian
  """
  p = d*1.0/(n - 1)
  G = nx.fast_gnp_random_graph(n, p, seed = 0, directed = False)

  # Random generator to choose random integer between emin and emax
  rndw = lambda : random.randrange(emin, emax + 1)
  h = { u: rndw() for u in range(n)}
  J = {(u, v): rndw() for u, v in G.edges()}
  return h, J

print("Generating random Ising")
h, J = generate_random_ising(4, 2, -4, 4)
print("h=", h,"\nJ=", J)

Generating random Ising
h= {0: 1, 1: -2, 2: -2, 3: -4, 4: 0, 5: 3, 6: -3, 7: -2} 
J= {(0, 4): -4, (0, 5): -3, (0, 7): 1, (1, 2): -3, (1, 3): 3, (1, 4): -1, (1, 5): 2, (2, 3): -2, (2, 4): 2, (2, 5): 1, (2, 6): 3, (2, 7): -2, (3, 6): -4, (3, 7): 1, (4, 5): -2, (4, 6): 2, (6, 7): -2}


In [8]:
# Solve the Ising using simulated annealing (SA)
sa = neal.SimulatedAnnealingSampler()
sample_set_sa = sa.sample_ising(h, J, num_reads = 100)
print(sample_set_sa.aggregate() )
print("Best solution has a minimum energy ", sample_set_sa.first.energy)


   0  1  2  3  4  5  6  7 energy num_oc.
0 -1 +1 +1 +1 -1 -1 +1 +1  -39.0     100
['SPIN', 1 rows, 100 samples, 8 variables]
Best solution has a minimum energy  -39.0


In [9]:
# Use FastHare composite to preprocess instance before sovling
# Solve the Ising using simulated annealing (SA)
fh_sa = FastHareComposite(sa)
sample_set_fh = fh_sa.sample_ising(h, J, num_reads = 100)
print(sample_set_fh.aggregate())
print("Best solution has a minimum energy ", sample_set_fh.first.energy)


FastHare: Reduced 3/8 variables (37.50% reduction).
   0  1  2  3  4  5  6  7 energy num_oc.
0 -1 +1 +1 +1 -1 -1 +1 +1  -39.0     100
['SPIN', 1 rows, 100 samples, 8 variables]
Best solution has a minimum energy  -39.0
